Before running this make the nov 2012 parquet with the vtec parquetcombiner

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.animation as animation
from skyfield.api import load, wgs84

# --- Load Data ---
df = pd.read_parquet("combined_ionex_vtecnov2012.parquet")
if not isinstance(df.columns, pd.MultiIndex):
    raise RuntimeError("Expected MultiIndex columns for [lat, lon]!")

lats = sorted(set(lat for lat, _ in df.columns), reverse=True)
lons = sorted(set(lon for _, lon in df.columns))
all_epochs = pd.to_datetime(df.index)
all_grids = [df.loc[epoch].values.reshape((len(lats), len(lons))) for epoch in all_epochs]
vmin = np.nanmin([np.nanmin(g) for g in all_grids])
vmax = np.nanmax([np.nanmax(g) for g in all_grids])

# --- Alaska bounding box ---
ALASKA_LON_MIN = -175
ALASKA_LON_MAX = -130
ALASKA_LAT_MIN = 51
ALASKA_LAT_MAX = 72

# --- Poker Flat location (add your marker here) ---
SITE_LAT = 64.794
SITE_LON = -147.536

# Optional: Setup Skyfield for the Moon position
ts = load.timescale()
eph = load('de421.bsp')
earth = eph['earth']
moon = eph['moon']

def get_moon_lonlat(dt):
    t = ts.utc(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
    subpoint = wgs84.subpoint(earth.at(t).observe(moon))
    lon = subpoint.longitude.degrees
    lat = subpoint.latitude.degrees
    # Wrap longitude to [-180, 180]
    if lon > 180: lon -= 360
    if lon < -180: lon += 360
    return lon, lat

fig = plt.figure(figsize=(10, 7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([ALASKA_LON_MIN, ALASKA_LON_MAX, ALASKA_LAT_MIN, ALASKA_LAT_MAX], ccrs.PlateCarree())
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
mesh = None

def update(frame_idx):
    global mesh
    ax.clear()
    ax.set_extent([ALASKA_LON_MIN, ALASKA_LON_MAX, ALASKA_LAT_MIN, ALASKA_LAT_MAX], ccrs.PlateCarree())
    ax.coastlines(resolution='10m')
    ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    
    epoch = all_epochs[frame_idx]
    grid = all_grids[frame_idx]
    
    mesh = ax.pcolormesh(
        lons, lats, grid,
        transform=ccrs.PlateCarree(),
        shading='auto', vmin=vmin, vmax=vmax, cmap='gist_ncar'
    )

    # Plot the Poker Flat location (red star, always shown)
    ax.plot(SITE_LON, SITE_LAT, marker='*', color='red', markersize=14, 
            markeredgecolor='black', markeredgewidth=1.5, 
            transform=ccrs.PlateCarree(), zorder=10)
    # Optional: label the site
    ax.text(SITE_LON+1, SITE_LAT, "OASIS STATION", color='red', fontsize=12,
            weight='bold', transform=ccrs.PlateCarree(), zorder=11)

    # Plot the Moon if it is over Alaska (optional)
    moon_lon, moon_lat = get_moon_lonlat(epoch)
    if (ALASKA_LON_MIN < moon_lon < ALASKA_LON_MAX) and (ALASKA_LAT_MIN < moon_lat < ALASKA_LAT_MAX):
        ax.plot(moon_lon, moon_lat, 'o', markersize=14, color='yellow',
                markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree(), label='Sublunar Point')
        ax.legend(loc='upper left')
    
    ax.set_title(f"VTEC over Alaska\n{epoch}")
    return mesh,

# --- Animation ---
ani = animation.FuncAnimation(fig, update, frames=len(df), interval=250, blit=False)
ani.save('alaska_vtec.gif', writer='pillow', fps=4)
print("Saved alaska_vtec.gif")

# --- Jupyter inline display (optional) ---
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())
